In [1]:
import tensorflow as tf

2022-08-25 15:06:25.207090: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-08-25 15:06:25.207117: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
mninst = tf.keras.datasets.mnist

(x_train,y_train),(x_test,y_test) = mninst.load_data()
sample = x_test[0]
x_train,x_test = x_train/255.0,x_test/255.0
#入力型を制限
x_train,x_test = x_train.astype("float32"),x_test.astype("float32")
#テストアイテムを捨てる
buf = tf.image.encode_png(sample[:, :, tf.newaxis])
tf.io.write_file("examples/mnist_savedmodel/sample.png", buf)

2022-08-25 15:15:46.015959: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-08-25 15:15:46.015998: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2022-08-25 15:15:46.016030: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (morio-H87M-S01): /proc/driver/nvidia/version does not exist
2022-08-25 15:15:46.016316: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [3]:
class classImage(tf.keras.Model):
  def __init__(self):
    super(classImage,self).__init__()
    self.flatten = tf.keras.layers.Flatten(input_shape=(28,28))
    self.dense1 = tf.keras.layers.Dense(128,activation="relu")
    self.drop = tf.keras.layers.Dropout(0.2)
    self.dense2 = tf.keras.layers.Dense(10)

  def call(self,x):
    x = self.flatten(x)
    x = self.dense1(x)
    x = self.drop(x)
    return self.dense2(x)

  
model = classImage()


In [5]:
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
model.compile(optimizer="adam",loss=loss_fn,metrics=["accuracy"])
model.fit(x_train,y_train,epochs=1)

2022-08-25 15:52:00.058834: W tensorflow/core/framework/cpu_allocator_impl.cc:80] Allocation of 188160000 exceeds 10% of free system memory.
2022-08-25 15:52:00.219940: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


1875/1875 [==============================] - 3s 1ms/step - loss: 0.3008 - accuracy: 0.9136


In [6]:
#確立として解釈できるように、ソフトマックスを介して出力タイプを変換
inputs = tf.keras.Input((28,28),name="input",dtype=tf.float32)
x = model(inputs)
outputs = tf.keras.layers.Softmax(name="output")(x)
probability_model = tf.keras.Model(inputs=inputs, outputs=outputs)

with open("examples/mnist_savedmodel/expected_values.txt", "w") as f:
    values = probability_model(x_test[:1, :, :])[0].numpy()
    print(*values, sep=", ", file=f)

In [7]:
directory = "examples/mnist_savedmodel"
tf.saved_model.save(probability_model, directory)

2022-08-25 16:10:04.834135: W tensorflow/python/util/util.cc:348] Sets are not currently considered sequences, but this may change in the future, so consider avoiding using them.


INFO:tensorflow:Assets written to: examples/mnist_savedmodel/assets
